In [40]:
from Bio import SeqIO
from Bio import Seq
import os

# Data preparation

In [105]:
# selecting and id change
for i in range(1, 11):
    records = list(SeqIO.parse("seqs/downloaded/" + str(i) + ".fasta", "fasta"))
    records = records[0:286]
    for j in range(0, len(records)):
        records[j].id = str(i) + "_" + str(j)
    SeqIO.write(records, "seqs/selected/" + str(i) + ".fasta", "fasta")

In [106]:
# merging
merged = list()
for i in range(1, 11):
    records = list(SeqIO.parse("seqs/selected/" + str(i) + ".fasta", "fasta"))
    merged = merged + records
    
SeqIO.write(merged, "seqs/merged.fasta", "fasta")

2860

In [107]:
len(merged)

2860

# CD-HIT

### running algorithm

In [116]:
binary_path = "../cd-hit-v4.8.1-2019-0228/cd-hit" # path to cd-hit executable
input_path = "seqs/merged.fasta"
output_path = "clustering_results/cd-hit/clusters"
threshold = 0.75
wordsize = 5

command = "./" + binary_path + " -i " + input_path + " -o " + output_path + \
" -c " + str(threshold) + " -n " + str(wordsize)

os.system(command)

0

### reading results

In [167]:
class Cluster:
    def __init__(self):
        self.elems = list()
        
    def add(self, id1, id2):
        self.elems.append((id1, id2))
    
    def ids1(self):
        res = list()
        for pair in self.elems:
            res.append(pair[0])
        return res
    
    def cleaned(self):
        cluster2 = Cluster()
        for elem in self.elems:
            if elem[0] not in cluster2.ids1():
                cluster2.add(elem[0], elem[1])
        return cluster2
    
    def idsfull(self):
        res = list()
        for pair in self.elems:
            res.append(str(pair[0]) + "_" + str(pair[1]))
        return res
    
        

In [168]:
clusters = list()
with open(output_path + ".clstr", 'r') as file:
    for line in file:
        if line.startswith(">"):
            clusters.append(Cluster())
        else:
            id1 = line[(line.index(">") + 1):line.index("_")]
            id2 = line[(line.index("_") + 1):line.index("...")]
            clusters[-1].add(int(id1), int(id2))

In [169]:
clusters2 = list()
for cluster in clusters:
    if all(x in cluster.ids1() for x in range(1, 11)):
        print(cluster.ids1())
        clusters2.append(cluster)

[1, 1, 1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10]
[1, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10]
[1, 2, 2, 2, 3, 3, 3, 4, 5, 5, 5, 6, 6, 6, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10]
[1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10]
[1, 1, 1, 1, 2, 2, 2, 3, 3, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 10, 10, 10]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10, 10]
[1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 9, 9, 9, 10, 10, 10]
[1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 5, 5, 5, 5, 6, 6, 6, 6, 7, 7, 7, 7, 8, 8, 8, 9, 

In [172]:
clusters3 = list()
for cluster in clusters2:
    filtered = cluster.cleaned()
    print(filtered.idsfull())
    clusters3.append(filtered)

['1_3', '2_13', '3_66', '4_63', '5_52', '6_7', '7_41', '8_28', '9_61', '10_60']
['1_208', '2_100', '3_67', '4_62', '5_51', '6_95', '7_42', '8_29', '9_62', '10_61']
['1_192', '2_114', '3_54', '4_86', '5_64', '6_1', '7_8', '8_43', '9_76', '10_75']
['1_114', '2_72', '3_31', '4_83', '5_85', '6_13', '7_9', '8_78', '9_37', '10_18']
['1_74', '2_92', '3_50', '4_42', '5_69', '6_80', '7_35', '8_17', '9_55', '10_54']
['1_36', '2_16', '3_49', '4_3', '5_70', '6_17', '7_33', '8_2', '9_4', '10_52']
['1_110', '2_103', '3_70', '4_59', '5_48', '6_37', '7_45', '8_32', '9_65', '10_64']
['1_111', '2_104', '3_71', '4_58', '5_47', '6_4', '7_46', '8_33', '9_66', '10_65']
['1_185', '2_121', '3_74', '4_48', '5_109', '6_119', '7_86', '8_23', '9_82', '10_81']
['1_196', '2_111', '3_57', '4_89', '5_61', '6_21', '7_60', '8_40', '9_73', '10_72']
['1_242', '2_68', '3_27', '4_79', '5_89', '6_3', '7_66', '8_82', '9_33', '10_22']
['1_243', '2_67', '3_26', '4_78', '5_90', '6_12', '7_67', '8_83', '9_32', '10_23']
['1_211',

In [173]:
cseqs = list()
for cluster in clusters3:
    cseq = list()
    for seq in merged:
        if seq.id in cluster.idsfull():
            cseq.append(seq)
    cseqs.append(cseq)

In [174]:
cseqs[0]

[SeqRecord(seq=Seq('IVVEMDPIKTSFEKWAKPGHFSXTLXKGPNTTTWIWNLHADAHDFGSHTNDLEE...GVA'), id='1_3', name='1_3', description='1_3 sp|Q9MUK0.1|PSAA_CYCRE RecName: Full=Photosystem I P700 chlorophyll a apoprotein A1; AltName: Full=PSI-A; AltName: Full=PsaA', dbxrefs=[]),
 SeqRecord(seq=Seq('VKIVVERDPIKTSFEKWAKPGHFSRTLAKGPNTTTWIWNLHADAHDFDSHTNDL...AHY'), id='2_13', name='2_13', description='2_13 sp|Q9MUC0.1|PSAA_GINBI RecName: Full=Photosystem I P700 chlorophyll a apoprotein A1; AltName: Full=PSI-A; AltName: Full=PsaA', dbxrefs=[]),
 SeqRecord(seq=Seq('MIMRSPESEVKIMVERDPIKTSFEKWANPGHFSRTLAKGDPETTTWIWNLHADA...AVE'), id='3_66', name='3_66', description='3_66 QJQ26551.1 photosystem I P700 apoprotein A1 (chloroplast) [Ephedra sinica]', dbxrefs=[]),
 SeqRecord(seq=Seq('MIIRPPEQPEVKVVVERDPVKTSFEKWAKPGHFSKTLAKGPETTTWIWNLHADA...AVG'), id='4_63', name='4_63', description='4_63 BAK86529.1 photosystem I P700 apoprotein A1 (chloroplast) [Larix decidua]', dbxrefs=[]),
 SeqRecord(seq=Seq('MIIRSPESEVKIVVEKDLIR